In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KernelDensity

def mesh_to_latlon(mesh_code):
    """1kmメッシュコード（3次メッシュ）の中心緯度経度を計算する"""
    if len(str(mesh_code)) != 8:
        raise ValueError("メッシュコードは8桁である必要があります。")
    mesh_code = str(mesh_code)
    m1_lat = int(mesh_code[0:2]) / 1.5
    m1_lon = int(mesh_code[2:4]) + 100
    m2_lat = int(mesh_code[4]) * 5 / 60
    m2_lon = int(mesh_code[5]) * 7.5 / 60
    m3_lat = int(mesh_code[6]) * 30 / 3600
    m3_lon = int(mesh_code[7]) * 45 / 3600
    lat = m1_lat + m2_lat + m3_lat + (30 / 3600) / 2
    lon = m1_lon + m2_lon + m3_lon + (45 / 3600) / 2
    return lat, lon

# 1. サンプルデータの作成
data = {
    'mesh_code': [
        '53393599', '53393690', '53394610', '53394611', '53393588',
        '53394509', '53394519', '53393671', '53393662', '53393652',
        '53394622', '53394632'
    ],
    'population': [150, 200, 180, 220, 130, 100, 120, 300, 250, 280, 190, 160]
}
df = pd.DataFrame(data)

# 2. 緯度経度への変換と重み付きデータ点の作成
df['lat'], df['lon'] = zip(*df['mesh_code'].apply(mesh_to_latlon))
points_weighted = np.repeat(df[['lat', 'lon']].values, df['population'], axis=0)

# -------------------------------------------------------------------
# 3. 最適なバンド幅の計算
# -------------------------------------------------------------------
n = len(points_weighted)
lats = points_weighted[:, 0]
lons = points_weighted[:, 1]

# 緯度 (latitude) のバンド幅を計算
std_lat = np.std(lats)
iqr_lat = np.percentile(lats, 75) - np.percentile(lats, 25)
# IQRが0の場合（データが単一値など）のエラーを回避
if iqr_lat == 0:
    iqr_lat = std_lat * 1.34 # IQRが0ならstdで代用
h_lat = 0.9 * min(std_lat, iqr_lat / 1.34) * n**(-1/5)

# 経度 (longitude) のバンド幅を計算
std_lon = np.std(lons)
iqr_lon = np.percentile(lons, 75) - np.percentile(lons, 25)
if iqr_lon == 0:
    iqr_lon = std_lon * 1.34
h_lon = 0.9 * min(std_lon, iqr_lon / 1.34) * n**(-1/5)

# 緯度と経度のバンド幅の平均値を最終的なバンド幅として採用
bandwidth = (h_lat + h_lon) / 2

print(f"データ点の数 (n): {n}")
print(f"計算されたバンド幅 (h): {bandwidth:.6f}")
# -------------------------------------------------------------------

# 4. カーネル密度推定の実行（計算したバンド幅を使用）
kde = KernelDensity(kernel='gaussian', bandwidth=bandwidth).fit(points_weighted)

# 5. 可視化のためのグリッド作成と密度計算
lat_min, lat_max = df['lat'].min(), df['lat'].max()
lon_min, lon_max = df['lon'].min(), df['lon'].max()
grid_size = 100
lat_grid = np.linspace(lat_min, lat_max, grid_size)
lon_grid = np.linspace(lon_min, lon_max, grid_size)
xx, yy = np.meshgrid(lon_grid, lat_grid)
grid_points = np.vstack([yy.ravel(), xx.ravel()]).T
log_density = kde.score_samples(grid_points)
density = np.exp(log_density)

# 6. QGIS用データ出力 (CSV)
qgis_df = pd.DataFrame({
    'latitude': grid_points[:, 0],
    'longitude': grid_points[:, 1],
    'density': density
})
qgis_df.to_csv("kernel_density_auto_bandwidth.csv", index=False)

print("\nQGIS用のデータを 'kernel_density_auto_bandwidth.csv' として保存しました。")

データ点の数 (n): 2280
計算されたバンド幅 (h): 0.003547

QGIS用のデータを 'kernel_density_auto_bandwidth.csv' として保存しました。
